９０．次単語予測

In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import torch.nn.functional as F

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

prompt = "The movie was full of"
input_ids = tokenizer.encode(prompt, return_tensors="pt")
print("トークン列:", tokenizer.convert_ids_to_tokens(input_ids[0]))

with torch.no_grad():
    outputs = model(input_ids)
    next_token_logits = outputs.logits[0, -1, :]
    probs = F.softmax(next_token_logits, dim=-1)
    top_k = torch.topk(probs, k=10)

top_tokens = tokenizer.convert_ids_to_tokens(top_k.indices.tolist())
top_probs = top_k.values.tolist()

for token, prob in zip(top_tokens, top_probs):
    print(f"{token}: {prob:.4f}")

トークン列: ['The', 'Ġmovie', 'Ġwas', 'Ġfull', 'Ġof']
Ġjokes: 0.0219
Ġgreat: 0.0186
Ġlaughs: 0.0115
Ġbad: 0.0109
Ġsurprises: 0.0107
Ġreferences: 0.0105
Ġfun: 0.0100
Ġhumor: 0.0074
Ġ": 0.0074
Ġthe: 0.0067


９１．続きのテキスト予測

In [5]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

prompt = "The movie was full of"
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# 温度パラメータや生成方式を変えてみる
for temp in [0.7, 1.0, 1.5]:
    print(f"\n--- Temperature: {temp} ---")
    output = model.generate(
        input_ids,
        max_length=30,
        do_sample=True,        # sampling を有効にする
        temperature=temp,
        top_k=50,              # トップK制御（任意）
        top_p=0.95,            # nucleus sampling（任意）
        num_return_sequences=1
    )
    print(tokenizer.decode(output[0], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



--- Temperature: 0.7 ---


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The movie was full of funny references that people were talking about. The joke I got was that he was like, "You've got to get out

--- Temperature: 1.0 ---


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The movie was full of jokes but the most telling aspect of this particular show was the way that it kept its emotional edge. "I love you,

--- Temperature: 1.5 ---
The movie was full of good things. And I'd rather have the bad stuff. (Laughs.)

Did anyone watch the film without reading about


９２．予測されたテキストの確率を計算

In [6]:
prompt = "The movie was full of"
input_ids = tokenizer.encode(prompt, return_tensors="pt")
output = model.generate(input_ids, max_length=12, do_sample=True)
generated_ids = output[0]

with torch.no_grad():
    outputs = model(generated_ids.unsqueeze(0))
    probs = F.softmax(outputs.logits, dim=-1)

for i in range(len(input_ids[0]), len(generated_ids)):
    token_id = generated_ids[i].item()
    prob = probs[0, i - 1, token_id].item()
    token = tokenizer.decode([token_id])
    print(f"{token}: {prob:.4f}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 fun: 0.0100
 moments: 0.0549
,: 0.3279
 and: 0.1136
 no: 0.0023
 one: 0.3667
 was: 0.1205


９３．パープレキシティ

In [7]:
from math import exp

def perplexity(sentence):
    input_ids = tokenizer.encode(sentence, return_tensors="pt")
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        loss = outputs.loss
    return exp(loss.item())

sentences = [
    "The movie was full of surprises",
    "The movies were full of surprises",
    "The movie were full of surprises",   # 文法誤り
    "The movies was full of surprises"    # 文法誤り
]

for s in sentences:
    ppl = perplexity(s)
    print(f"{s} -> Perplexity: {ppl:.2f}")


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


The movie was full of surprises -> Perplexity: 99.35
The movies were full of surprises -> Perplexity: 126.48
The movie were full of surprises -> Perplexity: 278.88
The movies was full of surprises -> Perplexity: 274.66


９４，チャットテンプレート

In [8]:
prompt = (
    "### Human:\n"
    "What do you call a sweet eaten after dinner?\n"
    "### Assistant:\n"
)

input_ids = tokenizer.encode(prompt, return_tensors="pt")
output = model.generate(input_ids, max_length=50, do_sample=True, temperature=0.9)
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


### Human:
What do you call a sweet eaten after dinner?
### Assistant:
### Human:
what does it say it says?
## Human:
what does it say it says about you?
Human:
How are
